<a href="https://colab.research.google.com/github/bhunivelze1/rice-quality-classification-model/blob/main/KlasifikasiKualitasBeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from skimage import data, io, feature, color, util, transform
import joblib

In [56]:
IMAGE_DIMENSION = 128
scaler = MinMaxScaler()

## Image Load

In [57]:
riceGoodPath = './drive/MyDrive/Dataset/Beras/Bagus/*.jpg'
riceMediumPath = './drive/MyDrive/Dataset/Beras/Sedang/*.jpg'
riceBadPath = './drive/MyDrive/Dataset/Beras/Buruk/*.jpg'

riceGoodImages = io.imread_collection(riceGoodPath)
riceMediumImages = io.imread_collection(riceMediumPath)
riceBadImages =io.imread_collection(riceBadPath)

print("JUMLAH GAMBAR")
print(f'Beras Bagus: {len(riceGoodImages)}')
print(f'Beras Sedang: {len(riceMediumImages)}')
print(f'Beras Buruk: {len(riceBadImages)}')
print(f'Total Gambar: {len(riceGoodImages) + len(riceMediumImages) + len(riceBadImages)} ')

JUMLAH GAMBAR
Beras Bagus: 491
Beras Sedang: 214
Beras Buruk: 204
Total Gambar: 909 


##Image Check

In [58]:
for index, item in enumerate(riceBadImages):
  if (item.shape[0] and item.shape[1]) != 128:
    print(index, ":", riceBadImages.files[index])

## Preprocessing

In [59]:
rice_good_image_list = []
rice_medium_image_list = []
rice_bad_image_list = []

rice_good_color_features = np.array([[]])
rice_good_shape_features = np.array([[]])
rice_good_texture_features = np.array([[]])

rice_medium_color_features = np.array([[]])
rice_medium_shape_features = np.array([[]])
rice_medium_texture_features = np.array([[]])

rice_bad_color_features = np.array([[]])
rice_bad_shape_features = np.array([[]])
rice_bad_texture_features = np.array([[]])

rice_good_image_data = np.array([[]])
rice_medium_image_data = np.array([[]])
rice_bad_image_data = np.array([[]])

for riceGoodImage in riceGoodImages:
  rice_good_image_list.append(riceGoodImage)

for riceMediumImage in riceMediumImages:
  rice_medium_image_list.append(riceMediumImage)

for riceBadImage in riceBadImages:
  rice_bad_image_list.append(riceBadImage)

## Feature Extraction

###Color

In [60]:
rice_good_hsv_image_first = color.rgb2hsv(rice_good_image_list[0])
rice_good_color_averaged_image_first = np.mean(rice_good_hsv_image_first, axis=2)
rice_good_flattened_image_first = rice_good_color_averaged_image_first.flatten()
rice_good_color_features = np.append(rice_good_color_features, [rice_good_flattened_image_first], axis=1)

for index in range(1, len(rice_good_image_list)):
  hsv_image = color.rgb2hsv(rice_good_image_list[index])
  color_averaged_image = np.mean(hsv_image, axis=2)
  flattened_image = color_averaged_image.flatten()
  rice_good_color_features = np.append(rice_good_color_features, [flattened_image], axis=0)

In [61]:
rice_medium_hsv_image_first = color.rgb2hsv(rice_medium_image_list[0])
rice_medium_color_averaged_image_first = np.mean(rice_medium_hsv_image_first, axis=2)
rice_medium_flattened_image_first = rice_medium_color_averaged_image_first.flatten()
rice_medium_color_features = np.append(rice_medium_color_features, [rice_medium_flattened_image_first], axis=1)

for index in range(1, len(rice_medium_image_list)):
  hsv_image = color.rgb2hsv(rice_medium_image_list[index])
  color_averaged_image = np.mean(hsv_image, axis=2)
  flattened_image = color_averaged_image.flatten()
  rice_medium_color_features = np.append(rice_medium_color_features, [flattened_image], axis=0)

In [62]:
rice_bad_hsv_image_first = color.rgb2hsv(rice_bad_image_list[0])
rice_bad_color_averaged_image_first = np.mean(rice_bad_hsv_image_first, axis=2)
rice_bad_flattened_image_first = rice_bad_color_averaged_image_first.flatten()
rice_bad_color_features = np.append(rice_bad_color_features, [rice_bad_flattened_image_first], axis=1)

for index in range(1, len(rice_bad_image_list)):
  hsv_image = color.rgb2hsv(rice_bad_image_list[index])
  color_averaged_image = np.mean(hsv_image, axis=2)
  flattened_image = color_averaged_image.flatten()
  rice_bad_color_features = np.append(rice_bad_color_features, [flattened_image], axis=0)

In [63]:
rice_good_color_features = scaler.fit_transform(rice_good_color_features)
rice_medium_color_features = scaler.fit_transform(rice_medium_color_features)
rice_bad_color_features = scaler.fit_transform(rice_bad_color_features)

In [64]:
print(f'Rice Good Color Features: {rice_good_color_features.shape}')
print(f'Rice Medium Color Features: {rice_medium_color_features.shape}')
print(f'Rice Bad Color Features: {rice_bad_color_features.shape}')

Rice Good Color Features: (491, 16384)
Rice Medium Color Features: (214, 16384)
Rice Bad Color Features: (204, 16384)


###Shape

In [65]:
rice_good_greyscale_canny_image_first = color.rgb2gray(rice_good_image_list[0])
rice_good_canny_image_first = feature.canny(rice_good_greyscale_canny_image_first)
rice_good_encoded_canny_image_first = rice_good_canny_image_first.astype(int)
rice_good_flattened_canny_image_first = rice_good_encoded_canny_image_first.flatten()
rice_good_shape_features = np.append(rice_good_shape_features, [rice_good_flattened_canny_image_first], axis=1)

for index in range(1, len(rice_good_image_list)):
  greyscale_canny_image = color.rgb2gray(rice_good_image_list[index])
  canny_image = feature.canny(greyscale_canny_image)
  encoded_canny_image = canny_image.astype(int)
  flattened_canny_image= encoded_canny_image.flatten()
  rice_good_shape_features = np.append(rice_good_shape_features, [flattened_canny_image], axis=0)



In [66]:
rice_medium_greyscale_canny_image_first = color.rgb2gray(rice_medium_image_list[0])
rice_medium_canny_image_first = feature.canny(rice_medium_greyscale_canny_image_first)
rice_medium_encoded_canny_image_first = rice_medium_canny_image_first.astype(int)
rice_medium_flattened_canny_image_first = rice_medium_encoded_canny_image_first.flatten()
rice_medium_shape_features = np.append(rice_medium_shape_features, [rice_medium_flattened_canny_image_first], axis=1)

for index in range(1, len(rice_medium_image_list)):
  greyscale_canny_image = color.rgb2gray(rice_medium_image_list[index])
  canny_image = feature.canny(greyscale_canny_image)
  encoded_canny_image = canny_image.astype(int)
  flattened_canny_image= encoded_canny_image.flatten()
  rice_medium_shape_features = np.append(rice_medium_shape_features, [flattened_canny_image], axis=0)

In [67]:
rice_bad_greyscale_canny_image_first = color.rgb2gray(rice_bad_image_list[0])
rice_bad_canny_image_first = feature.canny(rice_bad_greyscale_canny_image_first)
rice_bad_encoded_canny_image_first = rice_bad_canny_image_first.astype(int)
rice_bad_flattened_canny_image_first  = rice_bad_encoded_canny_image_first.flatten()
rice_bad_shape_features = np.append(rice_bad_shape_features, [rice_bad_flattened_canny_image_first], axis=1)

for index in range(1, len(rice_bad_image_list)):
  greyscale_canny_image = color.rgb2gray(rice_bad_image_list[index])
  canny_image = feature.canny(greyscale_canny_image)
  encoded_canny_image = canny_image.astype(int)
  flattened_canny_image= encoded_canny_image.flatten()
  rice_bad_shape_features = np.append(rice_bad_shape_features, [flattened_canny_image], axis=0)

In [68]:
print(f'Rice Good Shape Features: {rice_good_shape_features.shape}')
print(f'Rice Medium Shape Features: {rice_medium_shape_features.shape}')
print(f'Rice Bad Shape Features: {rice_bad_shape_features.shape}')

Rice Good Shape Features: (491, 16384)
Rice Medium Shape Features: (214, 16384)
Rice Bad Shape Features: (204, 16384)


### Texture

In [69]:
rice_good_greyscale_glcm_image_first = color.rgb2gray(rice_good_image_list[0])
rice_good_greyscale_glcm_image_first = util.img_as_ubyte(rice_good_greyscale_glcm_image_first)
rice_good_glcm_first = feature.greycomatrix(rice_good_greyscale_glcm_image_first, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4])

rice_good_prop_contrast_first = feature.greycoprops(rice_good_glcm_first, 'contrast')[0,0]
rice_good_prop_dissimilarity_first = feature.greycoprops(rice_good_glcm_first, 'dissimilarity')[0,0]
rice_good_prop_homogeneity_first = feature.greycoprops(rice_good_glcm_first, 'homogeneity')[0,0]
rice_good_prop_ASM_first = feature.greycoprops(rice_good_glcm_first, 'ASM')[0,0]
rice_good_prop_energy_first = feature.greycoprops(rice_good_glcm_first, 'energy')[0,0]
rice_good_prop_correlation_first = feature.greycoprops(rice_good_glcm_first, 'correlation')[0,0]

rice_good_texture_features_array_first = [rice_good_prop_contrast_first, rice_good_prop_dissimilarity_first, rice_good_prop_homogeneity_first,
                          rice_good_prop_ASM_first, rice_good_prop_energy_first, rice_good_prop_correlation_first]

rice_good_texture_features = np.append(rice_good_texture_features, [rice_good_texture_features_array_first], axis=1)

for index in range(1, len(rice_good_image_list)):
  greyscale_glcm_image = color.rgb2gray(rice_good_image_list[index])
  greyscale_glcm_image = util.img_as_ubyte(greyscale_glcm_image)
  glcm = feature.greycomatrix(greyscale_glcm_image, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4])

  prop_contrast = feature.greycoprops(glcm, 'contrast')[0,0]
  prop_dissimilarity = feature.greycoprops(glcm, 'dissimilarity')[0,0]
  prop_homogeneity = feature.greycoprops(glcm, 'homogeneity')[0,0]
  prop_ASM = feature.greycoprops(glcm, 'ASM')[0,0]
  prop_energy = feature.greycoprops(glcm, 'energy')[0,0]
  prop_correlation = feature.greycoprops(glcm, 'correlation')[0,0]

  texture_features_array = [prop_contrast, prop_dissimilarity, prop_homogeneity, prop_ASM, prop_energy, prop_correlation]

  rice_good_texture_features = np.append(rice_good_texture_features, [texture_features_array], axis=0)

In [70]:
rice_medium_greyscale_glcm_image_first = color.rgb2gray(rice_medium_image_list[0])
rice_medium_greyscale_glcm_image_first = util.img_as_ubyte(rice_medium_greyscale_glcm_image_first)
rice_medium_glcm_first = feature.greycomatrix(rice_medium_greyscale_glcm_image_first, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4])

rice_medium_prop_contrast_first = feature.greycoprops(rice_medium_glcm_first, 'contrast')[0,0]
rice_medium_prop_dissimilarity_first = feature.greycoprops(rice_medium_glcm_first, 'dissimilarity')[0,0]
rice_medium_prop_homogeneity_first = feature.greycoprops(rice_medium_glcm_first, 'homogeneity')[0,0]
rice_medium_prop_ASM_first = feature.greycoprops(rice_medium_glcm_first, 'ASM')[0,0]
rice_medium_prop_energy_first = feature.greycoprops(rice_medium_glcm_first, 'energy')[0,0]
rice_medium_prop_correlation_first = feature.greycoprops(rice_medium_glcm_first, 'correlation')[0,0]

rice_medium_texture_features_array_first = [rice_medium_prop_contrast_first, rice_medium_prop_dissimilarity_first, rice_medium_prop_homogeneity_first,
                          rice_medium_prop_ASM_first, rice_medium_prop_energy_first, rice_medium_prop_correlation_first]

rice_medium_texture_features = np.append(rice_medium_texture_features, [rice_medium_texture_features_array_first], axis=1)

for index in range(1, len(rice_medium_image_list)):
  greyscale_glcm_image = color.rgb2gray(rice_medium_image_list[index])
  greyscale_glcm_image = util.img_as_ubyte(greyscale_glcm_image)
  glcm = feature.greycomatrix(greyscale_glcm_image, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4])

  prop_contrast = feature.greycoprops(glcm, 'contrast')[0,0]
  prop_dissimilarity = feature.greycoprops(glcm, 'dissimilarity')[0,0]
  prop_homogeneity = feature.greycoprops(glcm, 'homogeneity')[0,0]
  prop_ASM = feature.greycoprops(glcm, 'ASM')[0,0]
  prop_energy = feature.greycoprops(glcm, 'energy')[0,0]
  prop_correlation = feature.greycoprops(glcm, 'correlation')[0,0]

  texture_features = [prop_contrast, prop_dissimilarity, prop_homogeneity, prop_ASM, prop_energy, prop_correlation]

  rice_medium_texture_features = np.append(rice_medium_texture_features, [texture_features], axis=0)

In [71]:
rice_bad_greyscale_glcm_image_first = color.rgb2gray(rice_bad_image_list[0])
rice_bad_greyscale_glcm_image_first = util.img_as_ubyte(rice_bad_greyscale_glcm_image_first)
rice_bad_glcm_first = feature.greycomatrix(rice_bad_greyscale_glcm_image_first, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4])

rice_bad_prop_contrast_first = feature.greycoprops(rice_bad_glcm_first, 'contrast')[0,0]
rice_bad_prop_dissimilarity_first = feature.greycoprops(rice_bad_glcm_first, 'dissimilarity')[0,0]
rice_bad_prop_homogeneity_first = feature.greycoprops(rice_bad_glcm_first, 'homogeneity')[0,0]
rice_bad_prop_ASM_first = feature.greycoprops(rice_bad_glcm_first, 'ASM')[0,0]
rice_bad_prop_energy_first = feature.greycoprops(rice_bad_glcm_first, 'energy')[0,0]
rice_bad_prop_correlation_first = feature.greycoprops(rice_bad_glcm_first, 'correlation')[0,0]

rice_bad_texture_features_first = [rice_bad_prop_contrast_first, rice_bad_prop_dissimilarity_first, rice_bad_prop_homogeneity_first,
                                   rice_bad_prop_ASM_first, rice_bad_prop_energy_first, rice_bad_prop_correlation_first]

rice_bad_texture_features = np.append(rice_bad_texture_features, [rice_bad_texture_features_first], axis=1)

for index in range(1, len(rice_bad_image_list)):
  greyscale_glcm_image = color.rgb2gray(rice_bad_image_list[index])
  greyscale_glcm_image = util.img_as_ubyte(greyscale_glcm_image)
  glcm = feature.greycomatrix(greyscale_glcm_image, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4])

  prop_contrast = feature.greycoprops(glcm, 'contrast')[0,0]
  prop_dissimilarity = feature.greycoprops(glcm, 'dissimilarity')[0,0]
  prop_homogeneity = feature.greycoprops(glcm, 'homogeneity')[0,0]
  prop_ASM = feature.greycoprops(glcm, 'ASM')[0,0]
  prop_energy = feature.greycoprops(glcm, 'energy')[0,0]
  prop_correlation = feature.greycoprops(glcm, 'correlation')[0,0]

  texture_features = [prop_contrast, prop_dissimilarity, prop_homogeneity, prop_ASM, prop_energy, prop_correlation]

  rice_bad_texture_features = np.append(rice_bad_texture_features, [texture_features], axis=0)

In [72]:
rice_good_texture_features = scaler.fit_transform(rice_good_texture_features)
rice_medium_texture_features = scaler.fit_transform(rice_medium_texture_features)
rice_bad_texture_features = scaler.fit_transform(rice_bad_texture_features)

In [73]:
print(f'Rice Good Texture Features: {rice_good_texture_features.shape}')
print(f'Rice Medium Texture Features: {rice_medium_texture_features.shape}')
print(f'Rice Bad Texture Features: {rice_bad_texture_features.shape}')

Rice Good Texture Features: (491, 6)
Rice Medium Texture Features: (214, 6)
Rice Bad Texture Features: (204, 6)


## Building Dataset

In [74]:
rice_good_image_data = np.hstack([rice_good_color_features, rice_good_shape_features, rice_good_texture_features])
rice_medium_image_data = np.hstack([rice_medium_color_features, rice_medium_shape_features, rice_medium_texture_features])
rice_bad_image_data = np.hstack([rice_bad_color_features, rice_bad_shape_features, rice_bad_texture_features])

### Feature Dataset

In [75]:
features_train = np.array([[]])
features_test = np.array([[]])

In [76]:
rice_good_features_train, rice_good_features_test = np.split(rice_good_image_data, [int(len(rice_good_image_data)*0.8)], axis=0)
rice_medium_features_train, rice_medium_features_test = np.split(rice_medium_image_data, [int(len(rice_medium_image_data)*0.8)], axis=0)
rice_bad_features_train, rice_bad_features_test = np.split(rice_bad_image_data, [int(len(rice_bad_image_data)*0.8)], axis=0)

In [77]:
features_train = np.vstack([rice_good_features_train, rice_medium_features_train, rice_bad_features_train])
features_test = np.vstack([rice_good_features_test, rice_medium_features_test, rice_bad_features_test])

### Label Dataset

In [78]:
labels_train = np.array([]).astype(np.int16)
labels_test = np.array([]).astype(np.int16)

In [79]:
rice_good_labels = np.array([]).astype(np.int16)
rice_medium_labels = np.array([]).astype(np.int16)
rice_bad_labels = np.array([]).astype(np.int16)

In [80]:
for x in range(len(rice_good_image_data)):
  rice_good_labels = np.append(rice_good_labels, 2)

for x in range(len(rice_medium_image_data)):
  rice_medium_labels = np.append(rice_medium_labels, 1)

for x in range(len(rice_bad_image_data)):
  rice_bad_labels = np.append(rice_bad_labels, 0)

In [81]:
rice_good_labels_train, rice_good_labels_test = np.split(rice_good_labels, [int(len(rice_good_labels)*0.8)], axis=0)
rice_medium_labels_train, rice_medium_labels_test = np.split(rice_medium_labels, [int(len(rice_medium_labels)*0.8)], axis=0)
rice_bad_labels_train, rice_bad_labels_test = np.split(rice_bad_labels, [int(len(rice_bad_labels)*0.8)], axis=0)

In [82]:
labels_train = np.hstack([rice_good_labels_train, rice_medium_labels_train, rice_bad_labels_train])
labels_test = np.hstack([rice_good_labels_test, rice_medium_labels_test, rice_bad_labels_test])

## Training & Testing Model

In [83]:
model = KNeighborsClassifier(n_neighbors=2)
model.fit(features_train, labels_train)
prediction_result = model.predict(features_test)
prediction_accuracy = metrics.accuracy_score(labels_test, prediction_result)
print(f'{round(prediction_accuracy * 100)}%')

80%


##Saving

In [84]:
model_filename = 'rice_classification_model.sav'
joblib.dump(model, model_filename)

['rice_classification_model.sav']